In [ ]:
import sys
import pandas as pd
import os
sys.path.append("..")

from drift_detector.reductor import ShiftReductor
from drift_detector.detector import ShiftDetector
from drift_detector.experiments import *
from utils.constants import *
from utils.plot_utils import errorfill, plot_roc, plot_pr, linestyles, markers, colors, brightness, colorscale
from utils.legacy_utils import import_dataset, import_dataset_hospital, FEATURES
from utils.utils import run_synthetic_shift_experiment
from baseline_models.static.utils import run_model

###  Load data ###

In [ ]:
print("Loading data...")  
(X_s_tr, y_s_tr), (X_s_val, y_s_val), (X_t, y_t), orig_dims = import_dataset('los', features=FEATURES)

### Define Shift Detector and Parameters ###

In [ ]:
sign_level = 0.05
dr_technique = "NoRed"
md_test = "Classifier"
red_model = None
dataset = "test experiment"
samples = [10, 20, 50, 100, 200, 500, 1000]
sample = samples[5]
orig_dims = X_s_tr.shape[1:]
print("Initializing shift reductor...")
shift_reductor = ShiftReductor(X_s_tr, y_s_tr, dr_technique, orig_dims, dataset, dr_amount=None, var_ret=0.9, scale=False, scaler="standard", model=None)
print("Initializing shift detector...")
shift_detector = ShiftDetector(dr_technique, md_test, sign_level, shift_reductor, sample, dataset)

#### Drift from 2019 to 2019 ####

In [ ]:
p_val, dist, t1_acc, t2_acc = shift_detector.detect_data_shift(X_s_tr, y_s_tr, X_s_tr, y_s_tr, X_s_val, y_s_val, orig_dims)
print(p_val,dist)

#### Drift from 2019 to 2020 ####

In [ ]:
p_val, dist, t1_acc, t2_acc = shift_detector.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_t, y_t, orig_dims)
print(p_val,dist)

#### Knockout Shift ####

In [ ]:
shift = "ko_shift_1.0"
X_ko, y_ko = apply_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, shift)
p_val, dist, t1_acc, t2_acc = shift_detector.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_ko, y_ko, orig_dims)
print(p_val,dist)

#### Gaussian Noise Shift ####

In [ ]:
shift = "small_gn_shift_0.1"
X_sgn, y_sgn = apply_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, shift)
p_val, dist, t1_acc, t2_acc = shift_detector.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_sgn, y_sgn, orig_dims)
print(p_val,dist)

shift = "large_gn_shift_1.0"
X_lgn, y_lgn = apply_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, shift)
p_val, dist, t1_acc, t2_acc = shift_detector.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_lgn, y_lgn, orig_dims)
print(p_val,dist)

#### Multiway Feature Association Shift ####

In [ ]:
shift = "mfa_shift_0.5"
X_mfa, y_mfa = apply_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, shift)
p_val, dist, t1_acc, t2_acc = shift_detector.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_mfa[:sample,:], y_mfa[:sample], orig_dims)
print(p_val,dist)

#### Changepoint Shift ####

In [ ]:
shift= "cp_shift_0.75"
X_cp, y_cp = apply_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, shift)
p_val, dist, t1_acc, t2_acc = shift_detector.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_cp[:sample,:], y_cp[:sample], orig_dims)
print(p_val,dist)